In [1]:
import pandas as pd
import mysql.connector
from getpass import getpass
from sqlalchemy import create_engine
import gdown

# Carga de datos en los datasets del database 'final_project' en MySQL Workbench

## Actualización del dataset 'accident'

In [90]:
# Carga el conjunto de datos del dataset accident
url = 'https://drive.google.com/uc?export=download&id=1CBC0qZL6u4awL0Z0hHpjwfKa6w3O6Yp2'

try: # Intenta ejecutar las líneas de código dentro de este bloque
    df_accident = pd.read_csv(url, encoding='latin1') # Lee el archivo CSV desde la URL especificada, utilizando el codec de caracteres 'latin1'
except UnicodeDecodeError as e: # Maneja la excepción UnicodeDecodeError si ocurre
    print(f"Error al decodificar el archivo CSV: {e}")  # Imprime un mensaje de error indicando el problema específico que ocurrió

In [91]:
df_accident['FATALS'].unique()

array([1, 2, 3, 5, 4, 6, 8, 9], dtype=int64)

In [92]:
# Imprime el dataframe y fuerza para que se muestren todas las columnas
pd.set_option('display.max_columns', None)
df_accident.head(1)

,STATE,STATENAME,ST_CASE,PEDS,PERNOTMVIT,VE_TOTAL,VE_FORMS,PVH_INVL,PERSONS,PERMVIT,COUNTY,COUNTYNAME,CITY,CITYNAME,MONTH,MONTHNAME,DAY,DAYNAME,DAY_WEEK,DAY_WEEKNAME,YEAR,HOUR,HOURNAME,MINUTE,MINUTENAME,TWAY_ID,TWAY_ID2,ROUTE,ROUTENAME,RUR_URB,RUR_URBNAME,FUNC_SYS,FUNC_SYSNAME,RD_OWNER,RD_OWNERNAME,NHS,NHSNAME,SP_JUR,SP_JURNAME,MILEPT,MILEPTNAME,LATITUDE,LATITUDENAME,LONGITUD,LONGITUDNAME,HARM_EV,HARM_EVNAME,MAN_COLL,MAN_COLLNAME,RELJCT1,RELJCT1NAME,RELJCT2,RELJCT2NAME,TYP_INT,TYP_INTNAME,REL_ROAD,REL_ROADNAME,WRK_ZONE,WRK_ZONENAME,LGT_COND,LGT_CONDNAME,WEATHER,WEATHERNAME,SCH_BUS,SCH_BUSNAME,RAIL,RAILNAME,NOT_HOUR,NOT_HOURNAME,NOT_MIN,NOT_MINNAME,ARR_HOUR,ARR_HOURNAME,ARR_MIN,ARR_MINNAME,HOSP_HR,HOSP_HRNAME,HOSP_MN,HOSP_MNNAME,FATALS
0,1,Alabama,10001,0,0,2,2,0,3,3,107,PICKENS (107),0,NOT APPLICABLE,1,January,1,1,7,Saturday,2022,12,12:00pm-12:59pm,30,30,US-82 SR-6,NaN,2,US Highway,1,Rural,3,Principal Arterial - Other,1,State Highway Agency,1,This section IS ON the NHS,0,No Special Jurisdiction,4,4,33.490967,33.490967,-88.274083,-88.274083,12,Motor Vehicle In-Transport,7,Sideswipe - Same Direction,0,No,1,Non-Junction,1,Not an Intersection,1,On Roadway,0,NaN,1,Daylight,1,Clear,0,No,0000000,Not Applicable,12,12:00pm-12:59pm,47,47,13,1:00pm-1:59pm,4,4,13,1:00pm-1:59pm,47,47,1


In [93]:
df_accident['FATALS'].unique()

array([1, 2, 3, 5, 4, 6, 8, 9], dtype=int64)

In [94]:
# Elimina las 47 columnas que no se necesitan
columns_to_drop = ['PEDS', 'VE_FORMS', 'MONTHNAME', 'DAYNAME', 'DAY_WEEKNAME', 'HOURNAME', 'MINUTENAME', 'TWAY_ID', 'TWAY_ID2', 'ROUTE', 'ROUTENAME', 'FUNC_SYS', 'FUNC_SYSNAME', 'NHS', 'NHSNAME', 'SP_JUR',\
                   'SP_JURNAME', 'MILEPT', 'MILEPTNAME', 'LATITUDENAME', 'LONGITUDNAME', 'MAN_COLLNAME', 'MAN_COLL', 'RELJCT1', 'RELJCT1NAME', 'RELJCT2', 'RELJCT2NAME', 'TYP_INT', 'TYP_INTNAME', 'REL_ROAD',\
                    'REL_ROADNAME', 'WRK_ZONE', 'WRK_ZONENAME', 'LGT_COND', 'LGT_CONDNAME', 'WEATHER', 'WEATHERNAME', 'SCH_BUS', 'SCH_BUSNAME', 'RAIL', 'RAILNAME', 'NOT_HOURNAME', 'NOT_MINNAME', 'ARR_HOURNAME',\
                        'ARR_MINNAME', 'HOSP_HRNAME', 'HOSP_MNNAME']
df_accident = df_accident.drop(columns=columns_to_drop)

In [95]:
# Reordena las columnas para que tengan el mismo orden que en la tabla creada en MySQL Workbench
new_column_order = ['ST_CASE', 'YEAR', 'MONTH', 'DAY', 'DAY_WEEK', 'HOUR', 'MINUTE', 'LATITUDE', 'LONGITUD', 'STATE', 'STATENAME', 'COUNTY', 'COUNTYNAME', 'CITY', 'CITYNAME', 'RUR_URB', 'RUR_URBNAME',\
      'RD_OWNER', 'RD_OWNERNAME', 'PERSONS', 'PERMVIT', 'PERNOTMVIT', 'VE_TOTAL', 'PVH_INVL', 'HARM_EV', 'HARM_EVNAME', 'FATALS', 'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN']

In [96]:
# Actualiza el dataframe con el nuevo orden
df_accident = df_accident[new_column_order]

In [97]:
# Mira la longitud máxima de la columna STATENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['STATENAME'].apply(len)
max_len = col_len.max()
max_len

20

In [98]:
# Mira la longitud máxima de la columna COUNTYNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['COUNTYNAME'].apply(len)
max_len = col_len.max()
max_len

28

In [99]:
# Mira la longitud máxima de la columna CITYNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['CITYNAME'].apply(len)
max_len = col_len.max()
max_len

39

In [100]:
# Mira la longitud máxima de la columna RUR_URBNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['RUR_URBNAME'].apply(len)
max_len = col_len.max()
max_len

33

In [101]:
# Mira la longitud máxima de la columna RD_OWNERNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['RD_OWNERNAME'].apply(len)
max_len = col_len.max()
max_len

44

In [102]:
# Mira la longitud máxima de la columna HARM_EVNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_accident['HARM_EVNAME'].apply(len)
max_len = col_len.max()
max_len

133

In [103]:
df_accident = df_accident.reset_index()

In [104]:
df_accident = df_accident.rename(columns={'index':'ID'})

In [106]:
df_accident.shape

(39221, 34)

In [19]:
# Datos para conectar MySQL
host = '127.0.0.1'  # La dirección IP de la máquina
database = 'final_project'  # El nombre de la base de datos
user = 'root'  # El nombre de usuario predeterminado de MySQL
port = '3306'
password = getpass("Introduce tu contraseña de MySQL: ")  # Solicita la contraseña de manera segura

In [20]:
# Crea la conexión a la base de datos MySQL utilizando la bublioteca SQLAlchemy
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

In [21]:
# Toma el dataframe df y lo mete dentro de la tabla ya creada en MySQL Workbench 'accident'
df_accident.to_sql('accident', engine, index=False, if_exists='append')

NameError: name 'df_accident' is not defined

In [242]:
df_accident_2 = pd.read_sql('SELECT * from final_project.accident', con=engine,dtype='object')

In [244]:
df_accident_2.isnull().sum()

ID                         0
ST_CASE                    0
YEAR                       0
MONTH                      0
DAY                        0
DAY_WEEK                   0
HOUR                       0
MINUTE                     0
LATITUDE                   0
LONGITUD                   0
STATE                      0
STATENAME                  0
COUNTY                     0
COUNTYNAME                 0
CITY                       0
CITYNAME                   0
RUR_URB                    0
RUR_URBNAME                0
RD_OWNER                   0
RD_OWNERNAME               0
PERSONS                    0
PERSONS_IN_VEHICLES        0
PERSONS_NOT_IN_VEHICLES    0
TOTAL_VEHICLES             0
PARKED_VEHICLES            0
HARMFUL_EVENT              0
HARMFUL_EVENT_NAME         0
FATALS                     0
NOT_HOUR                   0
NOT_MIN                    0
ARR_HOUR                   0
ARR_MIN                    0
HOSP_HR                    0
HOSP_MN                    0
dtype: int64

## Actualización del dataset 'drugs'

In [10]:
# Cargo el conjunto de datos del datset drugs
url = 'https://drive.google.com/uc?export=download&id=1jZbIeYuG1WVbM9VQZ7CGd5Zwo89pbnZy'

try: # Intenta ejecutar las líneas de código dentro de este bloque
    df_drugs = pd.read_csv(url, encoding='latin1') # Lee el archivo CSV desde la URL especificada, utilizando el codec de caracteres 'latin1'
except UnicodeDecodeError as e: # Maneja la excepción UnicodeDecodeError si ocurre
    print(f"Error al decodificar el archivo CSV: {e}")  # Imprime un mensaje de error indicando el problema específico que ocurrió

In [11]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME', 'DRUGSPEC', 'DRUGSPECNAME']
df_drugs = df_drugs.drop(columns=columns_to_drop)

In [12]:
# Mira la longitud máxima de la columna DRUGRESNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_drugs['DRUGRESNAME'].apply(len)
max_len = col_len.max()
max_len

100

In [13]:
# Reordena las columnas para que tengan el mismo orden que en la tabla creada en MySQL Workbench
new_column_order = ['ST_CASE', 'DRUGRES', 'DRUGRESNAME', 'VEH_NO', 'PER_NO']

In [14]:
# Actualiza el dataframe con el nuevo orden
df_drugs = df_drugs[new_column_order]

In [23]:
# Toma el dataframe df_drugs y lo mete dentro de la tabla ya creada en MySQL Workbench 'drugs'
df_drugs.to_sql('drugs', engine, index=False, if_exists='append')

120146

## Actualización del dataset 'maneuver'

In [135]:
# Cargo el conjunto de datos del datset maneuver
url = 'https://drive.google.com/uc?export=download&id=19FA4MUdhZwV2fIpZPWX73RQZS1xdnxpv'

try: # Intenta ejecutar las líneas de código dentro de este bloque
    df_maneuver = pd.read_csv(url, encoding='latin1') # Lee el archivo CSV desde la URL especificada, utilizando el codec de caracteres 'latin1'
except UnicodeDecodeError as e: # Maneja la excepción UnicodeDecodeError si ocurre
    print(f"Error al decodificar el archivo CSV: {e}")  # Imprime un mensaje de error indicando el problema específico que ocurrió

In [136]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME']
df_maneuver = df_maneuver.drop(columns=columns_to_drop)

In [137]:
# Mira la longitud máxima de la columna MANEUVERNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_maneuver['MANEUVERNAME'].apply(len)
max_len = col_len.max()
max_len

49

In [138]:
# Reordena las columnas para que tengan el mismo orden que en la tabla creada en MySQL Workbench
new_column_order = ['ST_CASE', 'MANEUVER', 'MANEUVERNAME', 'VEH_NO']

In [139]:
# Actualiza el dataframe con el nuevo orden
df_maneuver = df_maneuver[new_column_order]

In [140]:
# Toma el dataframe df_maneuver y lo mete dentro de la tabla ya creada en MySQL Workbench 'maneuver'
df_maneuver.to_sql('maneuver', engine, index=False, if_exists='append')

60513

## Actualización del dataset 'distract'

In [141]:
# Cargo el conjunto de datos del datset distract
url = 'https://drive.google.com/uc?export=download&id=1CG2v4hqWkmvGtN1pnXJOXfiq0T94PPe9'

try: # Intenta ejecutar las líneas de código dentro de este bloque
    df_distract = pd.read_csv(url, encoding='latin1') # Lee el archivo CSV desde la URL especificada, utilizando el codec de caracteres 'latin1'
except UnicodeDecodeError as e: # Maneja la excepción UnicodeDecodeError si ocurre
    print(f"Error al decodificar el archivo CSV: {e}")  # Imprime un mensaje de error indicando el problema específico que ocurrió

In [142]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME']
df_distract = df_distract.drop(columns=columns_to_drop)

In [143]:
# Mira la longitud máxima de la columna MANEUVERNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_distract['DRDISTRACTNAME'].apply(len)
max_len = col_len.max()
max_len

62

In [144]:
# Reordena las columnas para que tengan el mismo orden que en la tabla creada en MySQL Workbench
new_column_order = ['ST_CASE', 'DRDISTRACT', 'DRDISTRACTNAME', 'VEH_NO']

In [145]:
# Actualiza el dataframe con el nuevo orden
df_distract = df_distract[new_column_order]

In [146]:
# Toma el dataframe df_distract y lo mete dentro de la tabla ya creada en MySQL Workbench 'distract'
df_distract.to_sql('distract', engine, index=False, if_exists='append')

60525

## Actualización del dataset 'weather'

In [147]:
# Cargo el conjunto de datos del datset weather
url = 'https://drive.google.com/uc?export=download&id=1BKpyfiwu4f_V7IdK6jo3c8nDH5ca5FQM'

try: # Intenta ejecutar las líneas de código dentro de este bloque
    df_weather = pd.read_csv(url, encoding='latin1') # Lee el archivo CSV desde la URL especificada, utilizando el codec de caracteres 'latin1'
except UnicodeDecodeError as e: # Maneja la excepción UnicodeDecodeError si ocurre
    print(f"Error al decodificar el archivo CSV: {e}")  # Imprime un mensaje de error indicando el problema específico que ocurrió

In [148]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME']
df_weather = df_weather.drop(columns=columns_to_drop)

In [149]:
# Mira la longitud máxima de la columna WEATHERNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_weather['WEATHERNAME'].apply(len)
max_len = col_len.max()
max_len

24

In [150]:
# Toma el dataframe df_distract y lo mete dentro de la tabla ya creada en MySQL Workbench 'distract'
df_weather.to_sql('weather', engine, index=False, if_exists='append')

39644

## Actualización del dataset 'person'

In [74]:
# id del archivo en Google Drive
file_id = '1rRZEKB6OPGULbJVFhBoQF68TYTjKnreB'

# Construye la url para la descarga de datos
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Descarga el archivo con gdown
output = '../data/datain/person.csv'  # Especifica el camino donde quieres que se guarde el archivo
gdown.download(url, output, quiet=False)

# Lee el csv descargado en un dataframe
try:
    df_person = pd.read_csv(output, encoding='latin1')
    print(df_person.head())  # Muestra las primeras filas del dataframe
except UnicodeDecodeError as e:
    # Gestiona Unicode error si sucede
    print(f"Error decoding the CSV file: {e}")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1rRZEKB6OPGULbJVFhBoQF68TYTjKnreB
From (redirected): https://drive.google.com/uc?export=download&id=1rRZEKB6OPGULbJVFhBoQF68TYTjKnreB&confirm=t&uuid=976d8f2b-dbf2-45c5-823d-4e0980304177
To: c:\Users\perez\OneDrive\Documentos\GitHub\Proyectos\final_project\final_project\data\datain\person.csv
100%|██████████| 116M/116M [00:03<00:00, 37.1MB/s] 


   STATE STATENAME  ST_CASE  VEH_NO  PER_NO  VE_FORMS  COUNTY  MONTH  \
0      1   Alabama    10001       1       1         2     107      1   
1      1   Alabama    10001       2       1         2     107      1   
2      1   Alabama    10001       2       2         2     107      1   
3      1   Alabama    10002       1       1         2     101      1   
4      1   Alabama    10002       2       1         2     101      1   

  MONTHNAME  DAY  ...  IMPACT1     IMPACT1NAME FIRE_EXP        FIRE_EXPNAME  \
0   January    1  ...     81.0           Right      0.0  No or Not Reported   
1   January    1  ...      7.0   7 Clock Point      0.0  No or Not Reported   
2   January    1  ...      7.0   7 Clock Point      0.0  No or Not Reported   
3   January    1  ...     12.0  12 Clock Point      0.0  No or Not Reported   
4   January    1  ...     12.0  12 Clock Point      0.0  No or Not Reported   

   MAK_MOD                                        MAK_MODNAME DEVTYPENAME  \
0   2403.0  Jee

C:\Users\perez\AppData\Local\Temp\ipykernel_30332\2135696628.py:13: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_person = pd.read_csv(output, encoding='latin1')


In [75]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME', 'VE_FORMS', 'COUNTY', 'MONTH', 'MONTHNAME', 'DAY', 'DAYNAME', 'HOUR', 'HOURNAME', 'MINUTE', 'MINUTENAME', 'HARM_EV', 'HARM_EVNAME', 'MAN_COLL',\
                   'MAN_COLLNAME', 'SCH_BUS', 'SCH_BUSNAME', 'MOD_YEAR', 'MOD_YEARNAME', 'VPICMAKE', 'VPICMAKENAME', 'VPICMODEL', 'VPICMODELNAME', 'VPICBODYCLASS', 'VPICBODYCLASSNAME',\
                    'MAKE', 'MAKENAME', 'BODY_TYP', 'BODY_TYPNAME', 'ICFINALBODY', 'ICFINALBODYNAME', 'GVWR_FROM', 'GVWR_FROMNAME', 'GVWR_TO', 'GVWR_TONAME', 'TOW_VEH', 'TOW_VEHNAME',\
                        'AGENAME', 'SEX', 'REST_USE', 'REST_USENAME', 'HELM_USE', 'HELM_USENAME', 'EJ_PATH', 'EJ_PATHNAME', 'EXTRICAT', 'EXTRICATNAME', 'ALC_STATUS', 'ALC_STATUSNAME',\
                            'ATST_TYP', 'ATST_TYPNAME', 'ALC_RES', 'ALC_RESNAME', 'DRUGS', 'DRUGSNAME', 'DSTATUS', 'DSTATUSNAME', 'HOSPITAL', 'HOSPITALNAME', 'DOA', 'DOANAME', 'DEATH_DANAME',\
                                'DEATH_MO', 'DEATH_MONAME', 'DEATH_YR', 'DEATH_YRNAME', 'DEATH_HR', 'DEATH_HRNAME', 'DEATH_MN', 'DEATH_MNNAME', 'DEATH_TM', 'DEATH_TMNAME', 'LAG_HRS',\
                                    'LAG_HRSNAME', 'LAG_MINS', 'LAG_MINSNAME', 'HISPANIC', 'HISPANICNAME', 'IMPACT1NAME', 'FIRE_EXP', 'FIRE_EXPNAME', 'MAK_MOD', 'MAK_MODNAME', 'DEVTYPENAME',\
                                          'DEVMOTORNAME', 'DEVTYPE', 'DEVMOTOR', 'LOCATION', 'LOCATIONNAME', 'WORK_INJ', 'WORK_INJNAME', 'RUR_URB', 'RUR_URBNAME', 'FUNC_SYS', 'FUNC_SYSNAME',\
                                            'SPEC_USE', 'SPEC_USENAME', 'EMER_USE', 'EMER_USENAME', 'ROLLOVER', 'ROLLOVERNAME', 'IMPACT1', 'AIR_BAG', 'AIR_BAGNAME', 'EJECTION', 'EJECTIONNAME',\
                                                'STR_VEH']

df_person = df_person.drop(columns=columns_to_drop)

In [76]:
# Mira la longitud máxima de la columna SEXNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['SEXNAME'].apply(len)
max_len = col_len.max()
max_len

50

In [77]:
# Mira la longitud máxima de la columna PER_TYPNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['PER_TYPNAME'].apply(len)
max_len = col_len.max()
max_len

54

In [78]:
# Mira la longitud máxima de la columna INJ_SEVNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['INJ_SEVNAME'].apply(len)
max_len = col_len.max()
max_len

28

In [79]:
# Mira la longitud máxima de la columna SEAT_POSNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['SEAT_POSNAME'].apply(len)
max_len = col_len.max()
max_len

75

In [80]:
# Mira la longitud máxima de la columna REST_MISNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['REST_MISNAME'].apply(len)
max_len = col_len.max()
max_len

28

In [81]:
# Mira la longitud máxima de la columna HELM_MISNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['HELM_MISNAME'].apply(len)
max_len = col_len.max()
max_len

28

In [82]:
# Mira la longitud máxima de la columna DRINKINGNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_person['DRINKINGNAME'].apply(len)
max_len = col_len.max()
max_len

25

In [83]:
df_person

,ST_CASE,VEH_NO,PER_NO,AGE,SEXNAME,PER_TYP,PER_TYPNAME,INJ_SEV,INJ_SEVNAME,SEAT_POS,SEAT_POSNAME,REST_MIS,REST_MISNAME,HELM_MIS,HELM_MISNAME,DRINKING,DRINKINGNAME,DEATH_DA
0,10001,1,1,37,Male,1,Driver of a Motor Vehicle In-Transport,4,Fatal Injury (K),11,"Front Seat, Left Side",7,None Used/Not Applicable,7,None Used/Not Applicable,0,No (Alcohol Not Involved),1
1,10001,2,1,58,Female,1,Driver of a Motor Vehicle In-Transport,0,No Apparent Injury (O),11,"Front Seat, Left Side",0,No Indication of Misuse,7,None Used/Not Applicable,0,No (Alcohol Not Involved),88
2,10001,2,2,60,Female,2,Passenger of a Motor Vehicle In-Transport,0,No Apparent Injury (O),13,"Front Seat, Right Side",0,No Indication of Misuse,7,None Used/Not Applicable,8,Not Reported,88
3,10002,1,1,57,Female,1,Driver of a Motor Vehicle In-Transport,4,Fatal Injury (K),11,"Front Seat, Left Side",0,No Indication of Misuse,7,None Used/Not Applicable,9,Reported as Unknown,1
4,10002,2,1,55,Male,1,Driver of a Motor Vehicle In-Transport,4,Fatal Injury (K),11,"Front Seat, Left Side",0,No Indication of Misuse,7,None Used/Not Applicable,9,Reported as Unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95752,560117,1,1,43,Male,1,Driver of a Motor Vehicle In-Transport,4,Fatal Injury (K),11,"Front Seat, Left Side",7,None Used/Not Applicable,7,None Used/Not Applicable,1,Yes (Alcohol Involved),22
95753,560118,1,1,30,Male,1,Driver of a Motor Vehicle In-Transport,0,No Apparent Injury (O),11,"Front Seat, Left Side",7,None Used/Not Applicable,7,None Used/Not Applicable,1,Yes (Alcohol Involved),88
95754,560118,1,2,43,Female,2,Passenger of a Motor Vehicle In-Transport,4,Fatal Injury (K),23,"Second Seat, Right Side",7,None Used/Not Applicable,7,None Used/Not Applicable,8,Not Reported,17
95755,560118,1,3,30,Male,2,Passenger of a Motor Vehicle In-Transport,3,Suspected Serious Injury (A),21,"Second Seat, Left Side",7,None Used/Not Applicable,7,None Used/Not Applicable,8,Not Reported,88


In [160]:
# Toma el dataframe df_person y lo mete dentro de la tabla ya creada en MySQL Workbench 'person'
df_person.to_sql('person', engine, index=False, if_exists='append')

95757

## Actualización del dataset 'vehicle'

In [49]:
# id del archivo en Google Drive
file_id = '12fC_v0KQ6z4cBoZYVVqVfQPYsZyzSfMS'

# Construye la url para la descarga de datos
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Descarga el archivo con gdown
output = '../data/datain/vehicle.csv'  # Especifica el camino donde quieres que se guarde el archivo
gdown.download(url, output, quiet=False)

# Lee el csv descargado en un dataframe
try:
    df_vehicle = pd.read_csv(output, encoding='latin1')
    print(df_vehicle.head())  # Muestra las primeras filas del dataframe
except UnicodeDecodeError as e:
    # Gestiona Unicode error si sucede
    print(f"Error decoding the CSV file: {e}")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=12fC_v0KQ6z4cBoZYVVqVfQPYsZyzSfMS
From (redirected): https://drive.google.com/uc?export=download&id=12fC_v0KQ6z4cBoZYVVqVfQPYsZyzSfMS&confirm=t&uuid=e7ca9404-5572-4090-80de-21fe2466e9d9
To: c:\Users\perez\OneDrive\Documentos\GitHub\Proyectos\final_project\final_project\data\datain\vehicle.csv
100%|██████████| 113M/113M [00:02<00:00, 38.7MB/s] 
C:\Users\perez\AppData\Local\Temp\ipykernel_30332\3748650065.py:13: DtypeWarning: Columns (14,56,58,176,178) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicle = pd.read_csv(output, encoding='latin1')


   STATE STATENAME  ST_CASE  VEH_NO  VE_FORMS  NUMOCCS NUMOCCSNAME  DAY  \
0      1   Alabama    10001       1         2        1          01    1   
1      1   Alabama    10001       2         2        2          02    1   
2      1   Alabama    10002       1         2        1          01    1   
3      1   Alabama    10002       2         2        4          04    1   
4      1   Alabama    10003       1         1        2          02    1   

   DAYNAME  MONTH  ... GVWR_FROM  \
0        1      1  ...        11   
1        1      1  ...        11   
2        1      1  ...        11   
3        1      1  ...        11   
4        1      1  ...        11   

                                    GVWR_FROMNAME GVWR_TO  \
0  Class 1: 6,000 lbs. or less (2,722 kg or less)      11   
1  Class 1: 6,000 lbs. or less (2,722 kg or less)      11   
2  Class 1: 6,000 lbs. or less (2,722 kg or less)      11   
3  Class 1: 6,000 lbs. or less (2,722 kg or less)      11   
4  Class 1: 6,000 lbs. or l

In [50]:
# Elimina las columnas que no se necesitan
columns_to_drop = ['STATE', 'STATENAME', 'VE_FORMS', 'NUMOCCSNAME', 'DAY', 'DAYNAME', 'MONTH', 'MONTHNAME', 'HOUR', 'HOURNAME', 'MINUTE', 'MINUTENAME', 'HARM_EV', 'HARM_EVNAME', 'MAN_COLL',\
                   'MAN_COLLNAME', 'REG_STAT', 'REG_STATNAME', 'VIN', 'VINNAME', 'VIN_1', 'VIN_2', 'VIN_3', 'VIN_4', 'VIN_5', 'VIN_6', 'VIN_7', 'VIN_8', 'VIN_9', 'VIN_10', 'VIN_11', 'VIN_12',\
                    'J_KNIFE', 'J_KNIFENAME', 'MCARR_I1', 'MCARR_I1NAME', 'MCARR_I2', 'MCARR_I2NAME', 'MCARR_ID', 'MCARR_IDNAME', 'V_CONFIG', 'V_CONFIGNAME', 'CARGO_BT', 'CARGO_BTNAME',\
                        'HAZ_PLAC', 'HAZ_PLACNAME', 'HAZ_ID', 'HAZ_IDNAME', 'HAZ_CNO', 'HAZ_CNONAME', 'HAZ_REL', 'HAZ_RELNAME', 'TRAV_SPNAME', 'UNDEROVERRIDE', 'UNDEROVERRIDENAME', 'DEFORMED',\
                            'DEFORMEDNAME', 'L_STATE', 'L_STATENAME', 'DR_ZIP', 'DR_ZIPNAME', 'CDL_STAT', 'CDL_STATNAME', 'L_ENDORS', 'L_ENDORSNAME', 'L_STATUS', 'L_STATUSNAME', 'L_TYPE', 'L_TYPENAME',\
                                'DR_HGT', 'DR_HGTNAME', 'DR_WGT', 'DR_WGTNAME', 'PREV_OTH', 'PREV_OTHNAME', 'FIRST_MO', 'FIRST_MONAME', 'FIRST_YR', 'FIRST_YRNAME', 'LAST_MO', 'LAST_MONAME',\
                                    'LAST_YR', 'LAST_YRNAME', 'VTRAFWAY', 'VTRAFWAYNAME', 'VALIGN', 'VALIGNNAME', 'VPROFILE', 'VPROFILENAME', 'VPAVETYP', 'VPAVETYPNAME', 'VTRAFCON',\
                                          'VTRAFCONNAME', 'VTCONT_F', 'VTCONT_FNAME', 'TRLR1GVWR', 'TRLR1GVWRNAME', 'TRLR2GVWR', 'TRLR2GVWRNAME', 'TRLR3GVWR', 'TRLR3GVWRNAME', 'UNITTYPE', 'UNITTYPENAME',\
                                            'TRLR1VIN', 'TRLR1VINNAME', 'TRLR2VIN', 'TRLR2VINNAME', 'TRLR3VIN', 'TRLR3VINNAME', 'DR_DRINK', 'DR_DRINKNAME', 'MAKE', 'MAKENAME', 'MODEL', 'MAK_MOD',\
                                                'MAK_MODNAME', 'TOW_VEH', 'TOW_VEHNAME', 'VSPD_LIM', 'VSPD_LIMNAME', 'TRAV_SP', 'GVWR_TO', 'GVWR_TONAME', 'OWNER', 'OWNERNAME', 'VNUM_LAN', 'VNUM_LANNAME',\
                                                    'P_CRASH3', 'P_CRASH3NAME', 'PCRASH4', 'PCRASH4NAME', 'PCRASH5', 'PCRASH5NAME', 'P_CRASH1', 'P_CRASH1NAME', 'P_CRASH2', 'P_CRASH2NAME', 'ICFINALBODY',\
                                                        'ICFINALBODYNAME', 'MOD_YEARNAME', 'HAZ_INV', 'HAZ_INVNAME', 'DEATHS', 'FIRE_EXP', 'FIRE_EXPNAME', 'PREV_SUS2', 'PREV_SUS2NAME', 'PREV_SUS3',\
                                                            'PREV_SUS3NAME', 'PREV_ACCNAME', 'PREV_DWINAME', 'PREV_SPDNAME']

df_vehicle = df_vehicle.drop(columns=columns_to_drop)

In [51]:
# Reordena las columnas para que tengan el mismo orden que en la tabla creada en MySQL Workbench
new_column_order = ['ST_CASE', 'VEH_NO', 'BODY_TYP', 'BODY_TYPNAME', 'VPICMAKE', 'VPICMAKENAME', 'VPICMODEL', 'VPICMODELNAME', 'VPICBODYCLASS', 'VPICBODYCLASSNAME',\
                    'MOD_YEAR', 'GVWR_FROM', 'GVWR_FROMNAME', 'BUS_USE', 'BUS_USENAME', 'SPEC_USE', 'SPEC_USENAME', 'EMER_USE', 'EMER_USENAME', 'NUMOCCS',\
                        'ROLLOVER', 'ROLLOVERNAME', 'ROLINLOC', 'ROLINLOCNAME', 'IMPACT1', 'IMPACT1NAME', 'TOWED', 'TOWEDNAME', 'M_HARM', 'M_HARMNAME',\
                            'VSURCOND', 'VSURCONDNAME', 'ACC_TYPE', 'ACC_TYPENAME', 'DR_PRES', 'DR_PRESNAME', 'L_COMPL', 'L_COMPLNAME', 'L_RESTRI', 'L_RESTRINAME',\
                                'PREV_ACC', 'PREV_SUS1', 'PREV_DWI', 'PREV_SPD', 'SPEEDREL', 'SPEEDRELNAME',\
                                'HIT_RUN', 'HIT_RUNNAME']

In [52]:
# Actualiza el dataframe con el nuevo orden
df_vehicle = df_vehicle[new_column_order]

In [53]:
# Mira la longitud máxima de la columna BODY_TYPNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['BODY_TYPNAME'].apply(len)
max_len = col_len.max()
max_len

99

In [54]:
# Mira la longitud máxima de la columna VPICMAKENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['VPICMAKENAME'].apply(len)
max_len = col_len.max()
max_len

51

In [55]:
# Mira la longitud máxima de la columna VPICMODELNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['VPICMODELNAME'].apply(len)
max_len = col_len.max()
max_len

103

In [56]:
# Mira la longitud máxima de la columna VPICBODYCLASSNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['VPICBODYCLASSNAME'].apply(len)
max_len = col_len.max()
max_len

75

In [57]:
# Mira la longitud máxima de la columna GVWR_FROMNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['GVWR_FROMNAME'].apply(len)
max_len = col_len.max()
max_len

52

In [58]:
# Mira la longitud máxima de la columna BUS_USENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['BUS_USENAME'].apply(len)
max_len = col_len.max()
max_len

33

In [59]:
# Mira la longitud máxima de la columna SPEC_USENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['SPEC_USENAME'].apply(len)
max_len = col_len.max()
max_len

46

In [60]:
# Mira la longitud máxima de la columna EMER_USENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['EMER_USENAME'].apply(len)
max_len = col_len.max()
max_len

63

In [61]:
# Mira la longitud máxima de la columna ROLLOVERNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['ROLLOVERNAME'].apply(len)
max_len = col_len.max()
max_len

14

In [62]:
# Mira la longitud máxima de la columna ROLINLOCNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['ROLINLOCNAME'].apply(len)
max_len = col_len.max()
max_len

21

In [63]:
# Mira la longitud máxima de la columna IMPACT1NAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['IMPACT1NAME'].apply(len)
max_len = col_len.max()
max_len

61

In [64]:
# Mira la longitud máxima de la columna TOWEDNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['TOWEDNAME'].apply(len)
max_len = col_len.max()
max_len

19

In [65]:
# Mira la longitud máxima de la columna M_HARMNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['M_HARMNAME'].apply(len)
max_len = col_len.max()
max_len

133

In [66]:
# Mira la longitud máxima de la columna VSURCONDNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['VSURCONDNAME'].apply(len)
max_len = col_len.max()
max_len

33

In [67]:
# Mira la longitud máxima de la columna ACC_TYPENAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['ACC_TYPENAME'].apply(len)
max_len = col_len.max()
max_len

92

In [68]:
# Mira la longitud máxima de la columna DR_PRESNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['DR_PRESNAME'].apply(len)
max_len = col_len.max()
max_len

32

In [69]:
# Mira la longitud máxima de la columna L_COMPLNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['L_COMPLNAME'].apply(len)
max_len = col_len.max()
max_len

63

In [70]:
# Mira la longitud máxima de la columna L_RESTRINAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['L_RESTRINAME'].apply(len)
max_len = col_len.max()
max_len

43

In [71]:
# Mira la longitud máxima de la columna SPEEDRELNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['SPEEDRELNAME'].apply(len)
max_len = col_len.max()
max_len

43

In [72]:
# Mira la longitud máxima de la columna HIT_RUNNAME tipo texto para calcular el VARCHAR que hay que poner al crear la tabla
col_len = df_vehicle['HIT_RUNNAME'].apply(len)
max_len = col_len.max()
max_len

3

In [73]:
# Toma el dataframe df_vehicle y lo mete dentro de la tabla ya creada en MySQL Workbench 'vehicle'
df_vehicle.to_sql('vehicle', engine, index=False, if_exists='append')

60501